# Gemma-3-4b-it模型测试

In [ ]:
import transformers
import torch

model_id = "google/gemma-3-4b-it" 

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline([
        "一年有几个季节？分别是什么？",
         "小华有5个苹果，小明给了她3个，她又吃掉了2个，请问小华现在有几个苹果？",
    
    "一个水池有两个进水管和一个出水管。单开甲管8小时注满，单开乙管6小时注满，单开出水管12小时放完。如果三管同时开，多少小时能注满水池？",
    "鸡兔同笼，共有35个头，94只脚，请问鸡和兔各有多少只？",
    ])


/home/haoyiwen/miniconda3/envs/llama_factory/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.40it/s]
Device set to use cuda:0
/home/haoyiwen/miniconda3/envs/llama_factory/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


[[{'generated_text': '一年有几个季节？分别是什么？\n\n一年有四个季节：春、夏、秋、冬。\n\n*   **春季 (Spring)**：通常从3月到5月，气候温暖，植物开始生长。\n*   **夏季 (Summer)**：通常从6月到8月，气候炎热，阳光充足。\n*   **秋季 (Autumn/Fall)**：通常从9月到11月，气候凉爽，树叶变色。\n*   **冬季 (Winter)**：通常从12月到2月，气候寒冷，降雪多。\n\n当然，不同地区对季节的划分可能略有不同，例如，一些地方会将秋季分为早秋和晚秋。\n'}],
 [{'generated_text': '小华有5个苹果，小明给了她3个，她又吃掉了2个，请问小华现在有几个苹果？\n\n解：\n小华原来有5个苹果。\n小明给了她3个苹果，所以她有5+3=8个苹果。\n她又吃掉了2个苹果，所以她有8-2=6个苹果。\n答：小华现在有6个苹果。\n\n答案：6\n'}],
 [{'generated_text': '一个水池有两个进水管和一个出水管。单开甲管8小时注满，单开乙管6小时注满，单开出水管12小时放完。如果三管同时开，多少小时能注满水池？\n设水池的容量为V。\n单开甲管8小时注满，所以甲管的流量为 V/8。\n单开乙管6小时注满，所以乙管的流量为 V/6。\n单开出水管12小时放完，所以出水管的流量为 V/12。\n三管同时开，总流量为 (V/8) + (V/6) + (V/12)。\n为了合并这些分母，我们需要找到它们的最简公分母，LCM(8, 6, 12) = 24。\n所以，总流量为 (3V/24) + (4V/24) + (2V/24) = (3V + 4V + 2V) / 24 = 9V/24 = 3V/8。\n设t小时三管同时开，可以注满水池。\n那么，(3V/8) * t = V\nt = V / (3V/8) = V * (8/3V) = 8/3\n因此，三管同时开需要 8/'}],
 [{'generated_text': '鸡兔同笼，共有35个头，94只脚，请问鸡和兔各有多少只？\n\n这是一个经典的数学问题，可以用多种方法解决，这里提供两种常用的方法：\n\n**方法一：假设法**\n\n1. **假设全是鸡：** 如果35个头全是鸡，那么应该有 

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
测试加载训练好的模型
"""

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import warnings
warnings.filterwarnings("ignore")

def load_model_and_tokenizer(base_model_path="google/gemma-3-4b-it", 
                            lora_path="/home/haoyiwen/Documents/ai/LLaMA-Factory/saves/Gemma-3-4B-Instruct/lora"):
    """
    加载基础模型和LoRA适配器
    """
    print(f"加载基础模型: {base_model_path}")
    
    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_path,
        trust_remote_code=True,
        padding_side="left"
    )
    
    # 如果没有pad_token，设置为eos_token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 加载基础模型
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 加载LoRA适配器（如果存在）
    try:
        print(f"加载LoRA适配器: {lora_path}")
        model = PeftModel.from_pretrained(model, lora_path)
        print("LoRA适配器加载成功！")
    except Exception as e:
        print(f"注意: 未能加载LoRA适配器 - {e}")
        print("使用基础模型继续...")
    
    model.eval()
    return model, tokenizer

def generate_response(model, tokenizer, prompt, max_length=512):
    """
    生成模型响应
    """
    # 编码输入
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    
    # 移动到模型所在设备
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 生成响应
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解码输出
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def main():
    """
    主函数 - 测试模型
    """
    print("="*50)
    print("开始加载模型...")
    print("="*50)
    
    # 加载模型和tokenizer
    model, tokenizer = load_model_and_tokenizer()
    
    print("\n模型加载完成！开始测试...\n")
    print("="*50)
    
    # 测试示例
    test_prompts = [
        "一年有几个季节？分别是什么？",
         "小华有5个苹果，小明给了她3个，她又吃掉了2个，请问小华现在有几个苹果？",
    
    "一个水池有两个进水管和一个出水管。单开甲管8小时注满，单开乙管6小时注满，单开出水管12小时放完。如果三管同时开，多少小时能注满水池？",
    
    "鸡兔同笼，共有35个头，94只脚，请问鸡和兔各有多少只？",
    ]
    
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n测试 {i}:")
        print(f"输入: {prompt}")
        print("-"*30)
        
        response = generate_response(model, tokenizer, prompt)
        print(f"输出: {response}")
        print("="*50)
    
    # 交互式测试
    print("\n进入交互模式（输入'quit'退出）")
    print("-"*50)
    
    while True:
        user_input = input("\n请输入问题: ").strip()
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("退出程序")
            break
        
        if not user_input:
            continue
            
        print("\n生成中...")
        response = generate_response(model, tokenizer, user_input)
        print(f"\n模型回复: {response}")
        print("-"*50)

if __name__ == "__main__":
    main()

/home/haoyiwen/miniconda3/envs/llama_factory/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


开始加载模型...
加载基础模型: google/gemma-3-4b-it


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


加载LoRA适配器: /home/haoyiwen/Documents/ai/LLaMA-Factory/saves/Gemma-3-4B-Instruct/lora
注意: 未能加载LoRA适配器 - Can't find 'adapter_config.json' at '/home/haoyiwen/Documents/ai/LLaMA-Factory/saves/Gemma-3-4B-Instruct/lora'
使用基础模型继续...

模型加载完成！开始测试...


测试 1:
输入: 一年有几个季节？分别是什么？
------------------------------
输出: 一年有几个季节？分别是什么？

一年有四个季节：春、夏、秋、冬。

*   **春季**：通常在3月至5月之间，天气逐渐变暖，植物开始生长。
*   **夏季**：通常在6月至8月之间，天气炎热，阳光充足。
*   **秋季**：通常在9月至11月之间，天气凉爽，树叶变色。
*   **冬季**：通常在12月至2月之间，天气寒冷，降雪较多。

当然，不同地区的气候和季节划分可能略有差异。

希望以上信息对您有所帮助！

测试 2:
输入: 小华有5个苹果，小明给了她3个，她又吃掉了2个，请问小华现在有几个苹果？
------------------------------
输出: 小华有5个苹果，小明给了她3个，她又吃掉了2个，请问小华现在有几个苹果？

解题思路：

1.  计算小华得到苹果的数量：5 + 3 = 8
2.  计算小华吃掉苹果的数量：2
3.  计算小华剩余的苹果数量：8 - 2 = 6

答案：小华现在有6个苹果。


测试 3:
输入: 一个水池有两个进水管和一个出水管。单开甲管8小时注满，单开乙管6小时注满，单开出水管12小时放完。如果三管同时开，多少小时能注满水池？
------------------------------
输出: 一个水池有两个进水管和一个出水管。单开甲管8小时注满，单开乙管6小时注满，单开出水管12小时放完。如果三管同时开，多少小时能注满水池？

设水池的容量为V。
单开甲管8小时注满，所以甲管的流量为 V/8。
单开乙管6小时注满，所以乙管的流量为 V/6。
单开出水管12小时放完，所以出水管的流量为 V/12。

当三